# HOPP Example 01: Basic Hybrid Plant
Looks at a 50 MW plant (interconnection), with 50 MW of installed wind and 50 MW of installed solar.
The plant site is located at the NREL Flatirons.


In [1]:
import sys
import os
from pathlib import Path
import json
from pathlib import Path
import pandas as pd
sys.path.append('')

from hybrid.sites import SiteInfo, flatirons_site
from hybrid.hybrid_simulation import HybridSimulation
from hybrid.log import hybrid_logger as logger
from hybrid.keys import set_nrel_key_dot_env

examples_dir = Path("examples")

# Set API key
set_nrel_key_dot_env()

/mnt/c/Users/gstarke/Documents/Research_Programs/HOPP/HOPP/log/hybrid_systems_2023-01-23T17.07.34.499651.log


## Setting the plant characteristics

In [4]:
# Set characteristics of the system - NREL Flatirons site 

# Set wind, solar, and interconnection capacities (in MW)
solar_size_mw = 50
wind_size_mw = 50
interconnection_size_mw = 50

technologies = {'pv': {
                    'system_capacity_kw': solar_size_mw * 1000
                },
                'wind': {
                    'num_turbines': 10,
                    'turbine_rating_kw': 2000
                }}


## Loading the resource and site data

In [5]:
# Load the resource information
lat = flatirons_site['lat']
lon = flatirons_site['lon']
prices_file = examples_dir.parent / "resource_files" / "grid" / "pricing-data-2015-IronMtn-002_factors.csv"
site = SiteInfo(flatirons_site, grid_resource_file=prices_file)

## Running the hybrid plant

In [6]:
# Create model
hybrid_plant = HybridSimulation(technologies, site, interconnect_kw=interconnection_size_mw * 1000)

# Adjust some inptus to the model 
hybrid_plant.pv.system_capacity_kw = solar_size_mw * 1000
hybrid_plant.wind.system_capacity_by_num_turbines(wind_size_mw * 1000)
hybrid_plant.ppa_price = 0.1
hybrid_plant.pv.dc_degradation = [0] * 25

# Simulate the plant for 25 years
plant_life = 25
hybrid_plant.simulate(plant_life)

dispatch options:  None
Dispatch optimization not required...


## Writing the outputs from the simulation

In [6]:
# Save the outputs
annual_energies = hybrid_plant.annual_energies
wind_plus_solar_npv = hybrid_plant.net_present_values.wind + hybrid_plant.net_present_values.pv
npvs = hybrid_plant.net_present_values

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

print("Wind Installed Cost: {}".format(wind_installed_cost))
print("Solar Installed Cost: {}".format(solar_installed_cost))
print("Hybrid Installed Cost: {}".format(hybrid_installed_cost))
print("Wind NPV: {}".format(hybrid_plant.net_present_values.wind))
print("Solar NPV: {}".format(hybrid_plant.net_present_values.pv))
print("Hybrid NPV: {}".format(hybrid_plant.net_present_values.hybrid))
print("Wind + Solar Expected NPV: {}".format(wind_plus_solar_npv))


print(annual_energies)
print(npvs)

Wind Installed Cost: 72700000.0
Solar Installed Cost: 48000000.0
Hybrid Installed Cost: 120700000.0
Wind NPV: 57563093.90112695
Solar NPV: 42421773.179174826
Hybrid NPV: 92024910.99193442
Wind + Solar Expected NPV: 99984867.08030178
{"pv": 104053614.17242989, "wind": 154488744.90971112, "hybrid": 248440486.66069454}
{"pv": 42421773.179174826, "wind": 57563093.90112695, "hybrid": 92024910.99193442}
